LangChainの調査

In [18]:
import os
from dotenv import load_dotenv

load_dotenv('../.env')

True

In [ ]:
"""
model_name: str = "text-davinci-003"
    """Model name to use."""
    temperature: float = 0.7
    """What sampling temperature to use."""
    max_tokens: int = 256
    """The maximum number of tokens to generate in the completion.
    -1 returns as many tokens as possible given the prompt and
    the models maximal context size."""
    top_p: float = 1
    """Total probability mass of tokens to consider at each step."""
    frequency_penalty: float = 0
    """Penalizes repeated tokens according to frequency."""
    presence_penalty: float = 0
    """Penalizes repeated tokens."""
    n: int = 1
    """How many completions to generate for each prompt."""
    best_of: int = 1
    """Generates best_of completions server-side and returns the "best"."""
    model_kwargs: Dict[str, Any] = Field(default_factory=dict)
    """Holds any model parameters valid for `create` call not explicitly specified."""
    openai_api_key: Optional[str] = None
    openai_api_base: Optional[str] = None
    openai_organization: Optional[str] = None
    batch_size: int = 20
    """Batch size to use when passing multiple documents to generate."""
    request_timeout: Optional[Union[float, Tuple[float, float]]] = None
    """Timeout for requests to OpenAI completion API. Default is 600 seconds."""
    logit_bias: Optional[Dict[str, float]] = Field(default_factory=dict)
    """Adjust the probability of specific tokens being generated."""
    max_retries: int = 6
    """Maximum number of retries to make when generating."""
    streaming: bool = False
    """Whether to stream the results or not."""
    allowed_special: Union[Literal["all"], AbstractSet[str]] = set()
    """Set of special tokens that are allowed。"""
    disallowed_special: Union[Literal["all"], Collection[str]] = "all"
"""

In [14]:
# llms: OpenAIのラッパー
# OpenAIChatとの違い
from langchain.llms import OpenAI

llm = OpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.7,
    n=1,
)

text = "今の季節におすすめの料理は？"

llm(text)

'私はAIであるため、味わうことができないため、季節に合わせた料理のおすすめは提供できませんが、一般的に秋は栗やかぼちゃ、秋刀魚などがおすすめです。冬は鍋料理や煮込み料理、餅などが人気です。春は新鮮な野菜を使ったサラダや、桜餅などがおすすめです。夏は冷たい素麺や冷やし中華、かき氷などが人気です。'

In [17]:
from langchain.prompts import PromptTemplate
from langchain import LLMChain

prompt = PromptTemplate(
    input_variables=["product"],
    template="""
    Q: カレーの材料は？
    A: 材料:
    ・ジャガイモ
    ・にんじん
    ・牛肉
    ・カレー粉
    
    Q: {product}の材料は？
    A: 材料
    """
)

chain = LLMChain(llm=llm, prompt=prompt)

print(chain.run("もつ鍋"))

・もつ（豚の内臓）
    ・白菜
    ・にら
    ・にんにく
    ・生姜
    ・しょうゆ
    ・酒
    ・みりん
    ・水
    ・豆腐
    ・長ネギ


In [20]:
# 日本語に対応できてないし、安定性がイマイチ
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

llm = OpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
    n=1,
)
tools = load_tools(["serpapi"], llm=llm)
agent = initialize_agent(
    tools,
    llm,
    # https://book.st-hakky.com/docs/example-of-agents-of-langchain/
    agent="zero-shot-react-description",
    verbose=True
)

text1 = "大谷選手の最近の成績は？"
text2 = "大谷選手の最近の打率は？"

In [21]:
agent.run(text1)



> Entering new AgentExecutor chain...
I'm not sure what 大谷選手 means, I should look it up.
Action: Search
Action Input: "大谷選手 最近の成績"
Observation: 大谷翔平の最新打撃成績、速報、成績などを掲載 - 日刊スポーツ新聞社のニュースサイト、ニッカンスポーツ・コム（nikkansports.com）。
Thought:I found the information I needed, now I just need to translate it.
Action: Search
Action Input: "translate nikkansports.com"
Observation: Nikkan Sports is the first-launched Japanese daily sports newspaper founded in 1946. ... This article may be expanded with text translated from the corresponding ...
Thought:That wasn't helpful, I should try a different translation tool.
Action: Search
Action Input: "translate nikkansports.com with google translate"
Observation: Google's service, offered free of charge, instantly translates words, phrases, and web pages between English and over 100 other languages.
Thought:Great, now I can finally translate the article and find out 大谷選手's recent performance.
Action: Search
Action Input: "大谷翔平 最新打撃成績"
Observation: 大谷翔平の最新打撃成績、速

"According to nikkansports.com, 大谷翔平's recent batting performance is posted on their website."

In [ ]:
# エラーが発生
agent.run(text2)

In [23]:
from langchain import OpenAI, ConversationChain

llm = OpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
    n=1,
)
conversation = ConversationChain(llm=llm, verbose=True)

output = conversation.predict(input="おはよう！")
print(output)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: おはよう！
AI:

> Finished chain.
おはようございます！朝ですね。今日は何をしますか？(Good morning! It's morning. What are you going to do today?)


In [24]:
output = conversation.predict(input="おすすめはありますか？")
print(output)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: おはよう！
AI: おはようございます！朝ですね。今日は何をしますか？(Good morning! It's morning. What are you going to do today?)
Human: おすすめはありますか？
AI:

> Finished chain.
はい、たとえば、天気が良ければ公園に行くことをお勧めします。また、新しいレストランを試すこともできます。あなたの好みに合わせて、いくつかのオプションを提供することができます。(Yes, for example, if the weather is nice, I recommend going to the park. You could also try a new restaurant. I can provide several options based on your preferences.)


In [39]:
from langchain.prompts import (
    ChatPromptTemplate, 
    MessagesPlaceholder, 
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate
)
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

chat = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        "語尾に「ごわす」という言葉をつけて喋ってください"),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=chat, verbose=True)

In [40]:
conversation.predict(input="おはよう！")



> Entering new ConversationChain chain...
Prompt after formatting:
System: 語尾に「ごわす」という言葉をつけて喋ってください
Human: おはよう！

> Finished chain.


'おはようございます、ごわす！'

In [41]:
conversation.predict(input="朝におすすめの音楽はありますか？")



> Entering new ConversationChain chain...
Prompt after formatting:
System: 語尾に「ごわす」という言葉をつけて喋ってください
Human: おはよう！
AI: おはようございます、ごわす！
Human: 朝におすすめの音楽はありますか？

> Finished chain.


'朝に聴くと良いと思われる音楽は、ジャンルによって異なりますが、私がおすすめするのは、ジャズやクラシック、または穏やかなポップスなどです。例えば、ビル・エヴァンスの「Sunday at the Village Vanguard」や、ショパンの「ノクターン第2番」、ジョン・レジェンドの「All of Me」などがあります。ご参考までに、ごわす。'

In [42]:
conversation.predict(input="他におすすめはありますか？")



> Entering new ConversationChain chain...
Prompt after formatting:
System: 語尾に「ごわす」という言葉をつけて喋ってください
Human: おはよう！
AI: おはようございます、ごわす！
Human: 朝におすすめの音楽はありますか？
AI: 朝に聴くと良いと思われる音楽は、ジャンルによって異なりますが、私がおすすめするのは、ジャズやクラシック、または穏やかなポップスなどです。例えば、ビル・エヴァンスの「Sunday at the Village Vanguard」や、ショパンの「ノクターン第2番」、ジョン・レジェンドの「All of Me」などがあります。ご参考までに、ごわす。
Human: 他におすすめはありますか？

> Finished chain.


'はい、朝に聴くと良いと思われる音楽として、以下の曲もおすすめです。\n\n- ボサノバ：アストラッド・ジルベルトの「The Girl from Ipanema」\n- クラシック：モーツァルトの「アイネ・クライネ・ナハトムジーク」\n- ジャズ：チック・コリアの「Spain」\n- ポップス：ビートルズの「Here Comes the Sun」\n- インストゥルメンタル：石川さゆりの「津軽海峡・冬景色」のカバー\n\n朝の気分に合わせて、お好みの曲を選んで聴いてみてください。ごわす。'

In [44]:
# ドキュメントローダーをインポート
from langchain.document_loaders import TextLoader
# ドキュメントローダーの初期化
loader = TextLoader("../data/人間失格.txt")
documents = loader.load()

In [45]:
# チャンクサイズの制限を下回るまで再帰的に分割するテキストスプリッターのインポート
from langchain.text_splitter import RecursiveCharacterTextSplitter

# テキストスプリッターの初期化
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=0)

# テキストをチャンクに分割
texts = text_splitter.split_documents(documents)

In [47]:
# 使用するエンベッディングをインポート
from langchain.embeddings import OpenAIEmbeddings
# エンベッディングの初期化
embeddings = OpenAIEmbeddings()

In [49]:
# vectorstore をインポート (ここでは Chroma を使用)
from langchain.vectorstores import Chroma

# ベクターストアにドキュメントとエンベッディングを格納
db = Chroma.from_documents(texts, embeddings)

Using embedded DuckDB without persistence: data will be transient


In [51]:
retriever = db.as_retriever()

In [54]:
# 質問と回答の取得に使用するチェーンをインポート
from langchain.chains import RetrievalQA

# チェーンを作り、それを使って質問に答える
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

In [60]:
query = "主人公の見た目は？"
qa.run(query)

'文章には複数の主人公が登場していますが、それぞれの描写が異なります。最初の文章では、人間のからだに駄馬の首でもくっつけたような「死相」を持った男性の描写があります。第二の文章では、学生時代の写真が描写され、美貌の学生であるが、生きている人間の感じはしなかったという描写があります。最後の文章では、主人公の見た目については描写されていません。'

In [61]:
query = "主人公は何に苦しんでいた？"
qa.run(query)

'主人公は父親の死による苦悩や、自分自身の存在意義についての苦悩を抱えていました。また、金銭的な問題にも悩まされていました。'

In [68]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.llms import OpenAI

chat = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

qa_index_tool = Tool(
    name="QA Index"
    , description="人間失格に関するindex"
    , func=qa.run
)

agent_executor = initialize_agent(
    tools=[
        qa_index_tool
    ]
    , llm=chat
    , agent="conversational-react-description"
    , verbose=True
    , memory=memory
)

In [69]:
query = "どういう作品か教えて"
agent_executor.run(query)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: 田山花袋の小説『人間失格』は、主人公の自堕落な生活と、その生活によって引き起こされる様々な問題を描いた作品です。主人公の孤独や絶望感、そして自己嫌悪が強く描かれており、読者に深い印象を与える作品となっています。また、田山花袋の独特な文体や、哲学的なテーマも注目されています。

> Finished chain.


'田山花袋の小説『人間失格』は、主人公の自堕落な生活と、その生活によって引き起こされる様々な問題を描いた作品です。主人公の孤独や絶望感、そして自己嫌悪が強く描かれており、読者に深い印象を与える作品となっています。また、田山花袋の独特な文体や、哲学的なテーマも注目されています。'

In [ ]:
# 動作が安定せずにエラーが発生する
query = "自堕落な生活ってどんなの？"
agent_executor.run(query)